In [67]:
import pandas as pd 
import numpy as np
import yfinance as yf
import pandas_market_calendars as mcal
from datetime import datetime
import matplotlib.pyplot as plt

In [45]:
start = '2024-01-01'
end = '2024-01-31'
df = yf.download('AAPL', start=start, end=end, interval="1h")
df.index = pd.DatetimeIndex(df.index.strftime("%Y-%m-%d %H:%M"))
df.index

[*********************100%%**********************]  1 of 1 completed


DatetimeIndex(['2024-01-02 09:30:00', '2024-01-02 10:30:00',
               '2024-01-02 11:30:00', '2024-01-02 12:30:00',
               '2024-01-02 13:30:00', '2024-01-02 14:30:00',
               '2024-01-02 15:30:00', '2024-01-03 09:30:00',
               '2024-01-03 10:30:00', '2024-01-03 11:30:00',
               ...
               '2024-01-29 13:30:00', '2024-01-29 14:30:00',
               '2024-01-29 15:30:00', '2024-01-30 09:30:00',
               '2024-01-30 10:30:00', '2024-01-30 11:30:00',
               '2024-01-30 12:30:00', '2024-01-30 13:30:00',
               '2024-01-30 14:30:00', '2024-01-30 15:30:00'],
              dtype='datetime64[ns]', name='Datetime', length=140, freq=None)

In [47]:
# def combine_oc(df):
#     oc = []
#     for i in range(len(df)):
#         oc.append(df.iloc[i,0])
#         oc.append(df.iloc[i,2])
#     return oc

# %timeit combine_oc(df)


In [68]:
holidays = mcal.get_calendar('NYSE').holidays()
holidays = list(holidays.holidays) # NYSE Holidays

In [100]:
day_diff = np.busday_count(pd.to_datetime(start).date(), pd.to_datetime(end).date(), holidays=holidays)
oc = np.zeros((len(df) + day_diff, ))
# open values
oc[::8] = df.loc[::7, "Open"]
# close values
idx = np.ones((len(oc), ), dtype=bool)
idx[::8] = False
oc[idx] = df.loc[:, "Adj Close"]

In [101]:
# setting index for middle of day
idx = np.ones((len(oc), ), dtype=bool)
idx[7::8] = False
dates = np.zeros(oc.shape, dtype="datetime64[ns]")
dates[idx] = df.index
# setting index for end of day
later_index = df.index + pd.Timedelta(minutes=30)
dates[~idx] = later_index[6::7]
new_df = pd.DataFrame(data=oc, index=dates, columns=["Price"])